In [ ]:
## Run this command for GPU

#from tensorflow.compat.v1 import ConfigProto
#from tensorflow.compat.v1 import InteractiveSession

#config = ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

In [1]:
# Convolutional Neural Network
import tensorflow as tf

In [2]:
# Check TensorFlow version
tf.__version__

'2.3.1'

In [3]:
# Part 1 - Data Preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
training_set = train_datagen.flow_from_directory('datasets/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_set = test_datagen.flow_from_directory('datasets/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8029 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [5]:
# Importing the libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [6]:
# Part 2 - Building the CNN

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
classifier.add(MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
classifier.add(Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
classifier.add(MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full Connection
classifier.add(Dense(units=128, activation='relu'))

# Step 5 - Output Layer
classifier.add(Dense(units=1, activation='sigmoid'))

In [7]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [8]:
# Part 3 - Training the CNN

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
classifier.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
251/251 [==============================] - 74s 297ms/step - loss: 0.6827 - accuracy: 0.5536 - val_loss: 0.6526 - val_accuracy: 0.6185
Epoch 2/10
251/251 [==============================] - 74s 294ms/step - loss: 0.6168 - accuracy: 0.6567 - val_loss: 0.5844 - val_accuracy: 0.6985
Epoch 3/10
251/251 [==============================] - 69s 277ms/step - loss: 0.5745 - accuracy: 0.6988 - val_loss: 0.5915 - val_accuracy: 0.6725
Epoch 4/10
251/251 [==============================] - 70s 277ms/step - loss: 0.5417 - accuracy: 0.7271 - val_loss: 0.5221 - val_accuracy: 0.7495
Epoch 5/10
251/251 [==============================] - 76s 303ms/step - loss: 0.5165 - accuracy: 0.7426 - val_loss: 0.5102 - val_accuracy: 0.7555
Epoch 6/10
251/251 [==============================] - 77s 305ms/step - loss: 0.4920 - accuracy: 0.7595 - val_loss: 0.5104 - val_accuracy: 0.7470
Epoch 7/10
251/251 [==============================] - 82s 327ms/step - loss: 0.4773 - accuracy: 0.7683 - val_loss: 0.4758 - val_ac

In [9]:
# save it as a h5 file

from tensorflow.keras.models import load_model

classifier.save('model_rcat_dog.h5')

In [10]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('model_rcat_dog.h5')

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1048704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [12]:
# Part 4 - Making a single prediction

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('datasets/single_prediction/cat_or_dog_1.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)

In [13]:
result

array([[0.99860996]], dtype=float32)

In [14]:
if result[0]<=0.5:
    print("The image classified is cat")
else:
    print("The image classified is dog")

The image classified is dog


In [ ]:
# For multi class problems few changes are needed

# Step 5 - Output Layer: Change units based on class and activation function as softmax
#cnn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compiling the CNN: Change loss function as categorical_crossentropy
# cnn_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])